In [24]:
import numpy as np
import pandas as pd
import quandl
import matplotlib.pyplot as plt
from scipy.optimize import minimize
%matplotlib inline 

In [25]:
#data de inicio de começo 
start = pd.to_datetime('2012-01-01')
end = pd.to_datetime('2017-01-01')

In [26]:
#tickers
aapl = quandl.get('WIKI/AAPL.11',start_date=start,end_date=end)
cisco = quandl.get('WIKI/CSCO.11',start_date=start,end_date=end)
ibm = quandl.get('WIKI/IBM.11',start_date=start,end_date=end)
amzn = quandl.get('WIKI/AMZN.11',start_date=start,end_date=end)

LimitExceededError: (Status 429) (Quandl Error QELx01) You have exceeded the anonymous user limit of 50 calls per day. To make more calls today, please register for a free Quandl account and then include your API key with your requests.

In [ ]:
#juntar todos os tickers
stocks = pd.concat([aapl,cisco],axis=1)
stocks.columns = ['aapl','cisco']

In [ ]:
#retorno log
log_ret=np.log(stocks/stocks.shift(1))

In [ ]:
num_ports = 5000

all_weights = np.zeros((num_ports,len(stocks.columns)))
ret_arr = np.zeros(num_ports)
vol_arr = np.zeros(num_ports)
sharpe_arr = np.zeros(num_ports)

for ind in range(num_ports):

    # Create Random Weights
    weights = np.array(np.random.random(2))

    # Rebalance Weights
    weights = weights / np.sum(weights)
    
    # Save Weights
    all_weights[ind,:] = weights

    # Expected Return
    ret_arr[ind] = np.sum((log_ret.mean() * weights) *252)

    # Expected Variance
    vol_arr[ind] = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))

    # Sharpe Ratio
    sharpe_arr[ind] = ret_arr[ind]/vol_arr[ind]

In [ ]:
def get_ret_vol_sr(weights):
    weights = np.array(weights)
    ret = np.sum(log_ret.mean() * weights) * 252
    vol = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))
    sr = ret/vol
    return np.array([ret,vol,sr])

In [ ]:
def neg_sharpe(weights):
    return get_ret_vol_sr(weights)[2] * -1 

In [ ]:
def check_sum(weights):
    #return 0 if the sum of the weights is 1
    return np.sum(weights) -1 

In [ ]:
cons = ({'type':'eq','fun': check_sum})

In [ ]:
bounds = ((0, 1), (0, 1))

In [ ]:

init_guess = [0.25,0.25]

In [ ]:
opt_results = minimize(neg_sharpe,init_guess,method='SLSQP',bounds=bounds,constraints=cons)

In [ ]:
opt_results

In [ ]:
#aqui está a array com os pesos da carteira que devemos ter para maximizar o valor do sharpe ratio
opt_results.x

In [ ]:
#a partir desta parte, é codigo para trabalhar a parte ilustrativa de forma a melhor visulizar os resultados 
get_ret_vol_sr(opt_results.x)

In [ ]:
x = get_ret_vol_sr(opt_results.x)[0]
y = get_ret_vol_sr(opt_results.x)[1]

In [ ]:
frontier_y= np.linspace(0,0.3,100)

In [ ]:
def minimize_vol(weights):
    return get_ret_vol_sr(weights)[1]

In [ ]:
frontier_volatility = []
for possible_return in frontier_y:
    cons = ({'type':'eq','fun':check_sum},
            {'type':'eq','fun':lambda w: get_ret_vol_sr(w)[0]-possible_return})
    result = minimize(minimize_vol,init_guess,method='SLSQP',bounds=bounds,constraints=cons)
    
    frontier_volatility.append(result['fun'])

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(vol_arr,ret_arr,c=sharpe_arr,cmap='plasma')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility')
plt.ylabel('Return')

plt.plot(frontier_volatility,frontier_y,'g--',linewidth=3,alpha=0.7)

plt.scatter(y,x,c='red',s=50,edgecolors='black')

print('Compra', opt_results.x[0] * 100,'% de Apple')
print('Compra', opt_results.x[1] * 100,'% de Cisco')
print('Compra', opt_results.x[2] * 100,'% de IBM')
print('Compra', opt_results.x[3] * 100,'% de Amazon')



In [22]:
type(aapl)


pandas.core.frame.DataFrame

In [23]:
print('Compra', opt_results.x[0] * 100,'% de Apple')
print('Compra', opt_results.x[1] * 100,'% de Cisco')
print('Compra', opt_results.x[2] * 100,'% de IBM')
print('Compra', opt_results.x[3] * 100,'% de Amazon')


Compra 26.62897679421034 % de Apple
Compra 20.41898363983549 % de Cisco
Compra 1.1816448427376391e-14 % de IBM
Compra 52.95203956595416 % de Amazon
